In [1]:
import pandas as pd
import pickle
from scipy.stats import poisson

In [5]:
dict_table = pickle.load(open('data/dict_table', 'rb'))
df_historical_data = pd.read_csv('data/fifa_worldcup_matches_clean.csv')
df_fixture = pd.read_csv('data/fifa_worldcup_fixture_clean.csv')

## 1. Calcular team strength (fuerza de un equipo)

In [6]:
dict_table

{'Group A':    Pos         Team  Pld  W  D  L  GF  GA  GD  Pts
 0    1    Qatar (H)    0  0  0  0   0   0   0    0
 1    2      Ecuador    0  0  0  0   0   0   0    0
 2    3      Senegal    0  0  0  0   0   0   0    0
 3    4  Netherlands    0  0  0  0   0   0   0    0,
 'Group B':    Pos           Team  Pld  W  D  L  GF  GA  GD  Pts
 0    1        England    0  0  0  0   0   0   0    0
 1    2           Iran    0  0  0  0   0   0   0    0
 2    3  United States    0  0  0  0   0   0   0    0
 3    4          Wales    0  0  0  0   0   0   0    0,
 'Group C':    Pos          Team  Pld  W  D  L  GF  GA  GD  Pts
 0    1     Argentina    0  0  0  0   0   0   0    0
 1    2  Saudi Arabia    0  0  0  0   0   0   0    0
 2    3        Mexico    0  0  0  0   0   0   0    0
 3    4        Poland    0  0  0  0   0   0   0    0,
 'Group D':    Pos       Team  Pld  W  D  L  GF  GA  GD  Pts
 0    1     France    0  0  0  0   0   0   0    0
 1    2  Australia    0  0  0  0   0   0   0    0
 2    3 

In [7]:
df_historical_data

,local_team,visit_team,year,local_goals,visit_goals,total_goals
0,Yugoslavia,Brazil,1930,2,1,3
1,Uruguay,Yugoslavia,1930,6,1,7
2,Argentina,United States,1930,6,1,7
3,Paraguay,Belgium,1930,1,0,1
4,United States,Paraguay,1930,3,0,3
...,...,...,...,...,...,...
895,Brazil,Costa Rica,2018,2,0,2
896,Serbia,Switzerland,2018,1,2,3
897,Serbia,Brazil,2018,0,2,2
898,Germany,Mexico,2018,0,1,1


In [16]:
# creamos dos dataframe para equipo local y equipo visitante
df_local = df_historical_data[['local_team', 'local_goals', 'visit_goals']]
df_visit = df_historical_data[['visit_team', 'local_goals', 'visit_goals']]

In [17]:
# renombramos las columnas
df_local = df_local.rename(columns={'local_team': 'team', 'local_goals': 'goals_scored', 'visit_goals': 'goals_conceded'})
df_visit = df_visit.rename(columns={'visit_team': 'team', 'local_goals': 'goals_conceded', 'visit_goals': 'goals_scored'})

In [23]:
df_local

,team,goals_scored,goals_conceded
0,Yugoslavia,2,1
1,Uruguay,6,1
2,Argentina,6,1
3,Paraguay,1,0
4,United States,3,0
...,...,...,...
895,Brazil,2,0
896,Serbia,1,2
897,Serbia,0,2
898,Germany,0,1


In [24]:
# unimos ambos, agrupamos por equipo y hallamos el promedio
df_team_strength = pd.concat([df_local, df_visit], ignore_index=True).groupby('team').mean()

In [28]:
df_team_strength

,goals_scored,goals_conceded
team,,
Algeria,0.500000,0.750000
Angola,0.500000,0.500000
Argentina,1.000000,1.571429
Australia,0.600000,2.200000
Austria,0.666667,0.888889
...,...,...
United States,1.111111,2.888889
Uruguay,0.615385,1.615385
Wales,0.500000,0.750000


## 2. Function predict_points

In [ ]:
def predict_points(local, visit):
    if local in df_team_strength.index and visit in df_team_strength.index:
        lamb_local = df_team_strength.at[local, 'goals_scored'] * df_team_strength[visit, 'goals_conceded']
        lamb_visit = df_team_strength.at[visit, 'goals_scored'] * df_team_strength[local, 'goals_conceded']
        prob_local, prob_visit, prob_draw = 0, 0, 0
        for x in range(0, 11):
            for y in range(0, 11):
                p = poisson.pmf(x, lamb_local) * poisson.pmf(y, lamb_visit)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_local += p
                else:
                    prob_visit += p

        points_local = 3 * prob_local + prob_draw
        points_visit = 3 * prob_visit + prob_draw

        return points_local, points_visit
    else:
        return 0, 0